# LOAD THE LIBRARIES

In [10]:
import pandas as pd
import numpy as np

#googlee play scrapper
from google_play_scraper import app
from google_play_scraper import Sort, reviews_all

#cleaning
import contractions

#VADER
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

#feature extraction
from keybert import KeyBERT
kw_model = KeyBERT()
from keyphrase_vectorizers import KeyphraseCountVectorizer

#language id
from langdetect import detect

# SCRAPING

In [2]:
reviews = reviews_all(
    'com.linkdokter.halodoc.android',
    sleep_milliseconds=0, # defaults to 0
    lang='en', # defaults to 'en'
    country='id', # defaults to 'us'
    sort=Sort.NEWEST, # defaults to Sort.MOST_RELEVANT
)

halodocraw = pd.DataFrame(np.array(reviews),columns=['review'])
halodocraw = halodocraw.join(pd.DataFrame(halodocraw.pop('review').tolist()))
halodocraw.to_csv('data_halodocraw.csv',index=False)
halodocraw

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,32967bba-83e5-4e2c-8562-a6a903f5ace2,Anonim saja,https://play-lh.googleusercontent.com/a/ALm5wu...,Tidak sengaja menekan lanjut konsul menggunaka...,1,0,15.1,2022-11-11 20:04:52,Hai Anonim saja! Mohon maaf atas ketidaknyaman...,2022-11-11 20:35:02
1,ac5d2533-08f2-45ca-b417-dcb83d42dc62,Priyo Sembodo,https://play-lh.googleusercontent.com/a/ALm5wu...,Di app ini sekarang tidak bisa isi saldo halod...,1,0,15.0,2022-11-11 19:19:34,Hai Priyo Sembodo! Mohon maaf atas ketidaknyam...,2022-11-11 20:25:50
2,eaa06536-375e-4fde-a259-e7094f34c1f9,sheren virginia,https://play-lh.googleusercontent.com/a-/ACNPE...,"good application, so helpfull",5,0,15.0,2022-11-11 19:13:44,Hai sheren virginia! Terima kasih sudah menggu...,2022-11-11 20:21:24
3,ab8880fe-76a0-4376-8558-55d84644df5d,ayudiahnkh 25,https://play-lh.googleusercontent.com/a-/ACNPE...,it helps a lot.,5,0,15.0,2022-11-11 17:25:41,Hai ayudiahnkh 25! Terima kasih sudah mengguna...,2022-11-11 17:38:32
4,1014712e-925a-4208-95fa-a0f8cc0d8271,Johannes Clinton,https://play-lh.googleusercontent.com/a-/ACNPE...,"Aduh gimana ini app parah banget, baru nyoba p...",1,0,15.0,2022-11-11 16:31:21,Hai Johannes Clinton! Mohon maaf atas ketidakn...,2022-11-11 15:45:19
...,...,...,...,...,...,...,...,...,...,...
32800,6be3f0f1-a715-4a4e-8d41-4c28a3dc0d51,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,It's very easy and helpful,5,2,None,2016-04-02 12:54:34,Hai! Terima kasih sudah menggunakan Halodoc. J...,2019-12-16 08:16:39
32801,15cad9f3-5d69-4b43-ba94-c5e788cf554b,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Sepertinya ini aplikasi pertama di Indonesia.....,5,4,0.16.2-build201603301019,2016-04-02 11:06:15,Hai! Terima kasih sudah menggunakan Halodoc. J...,2019-12-16 08:19:05
32802,67a73c69-81af-4839-9fee-813b4b53bbfb,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Good aplication,5,1,0.16.2-build201603301019,2016-04-02 11:02:41,Hai! Terima kasih sudah menggunakan Halodoc. J...,2019-12-16 08:19:03
32803,61db862b-82b4-4c73-997b-b6c833595f74,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,This is simplying health,5,1,0.13.1-build201603171038,2016-03-18 19:11:25,Hai! Terima kasih sudah menggunakan Halodoc. J...,2019-12-16 08:17:08


# DROP COLUMNS AND FILTER

In [9]:
halodocfiltered = pd.read_csv(r'data_halodocraw.csv')
halodocfiltered['at'] = pd.to_datetime(halodocfiltered['at'], format='%Y-%m-%d')
halodocfiltered = halodocfiltered.loc[(halodocfiltered['at'] >= "2022-1-1") & (halodocfiltered['at'] <= "2022-10-31")]
halodocfiltered = halodocfiltered.reset_index(drop=True)
halodocfiltered = halodocfiltered.drop(columns=['reviewId','userName','userImage','thumbsUpCount','reviewCreatedVersion','at','replyContent','repliedAt'])
halodocfiltered.to_csv('data_halodocfiltered.csv',index=False)
halodocfiltered

,content,score
0,Great app. Selama ini hanya dipakai untuk beli...,5
1,"i've already uploaded the medicine recipe, why...",1
2,Really helpfull,5
3,"barang kadaluarsa masih bisa dikirim, katanya ...",1
4,Nice application,5
...,...,...
3911,gercep,5
3912,"Puas konsul ama dokter enditiara, sukses selal...",5
3913,Bagus bagus bagus,5
3914,"Nice dok, thanks",5


# CLEAN & TRANS

In [16]:
halodoccleaned = pd.read_csv(r'data_halodocfiltered.csv')

#choose reviews with 2 words or more
halodoccleaned = halodoccleaned[[len(i.split())>2 for i in halodoccleaned['content'].values]]

#fix contractions
halodoccleaned['content_contractions'] = halodoccleaned['content'].apply(lambda x: contractions.fix(x))

#translate the reviews to english
#halodoccleaned['content_translated'] = halodoccleaned['content_contractions'].apply(lambda x: gs.translate(x, 'en'))
halodoccleaned.to_csv('data_halodoccleaned.csv',index=False)
halodoccleaned

,content,score,content_contractions
0,Great app. Selama ini hanya dipakai untuk beli...,5,Great app. Selama ini hanya dipakai untuk beli...
1,"i've already uploaded the medicine recipe, why...",1,"i have already uploaded the medicine recipe, w..."
3,"barang kadaluarsa masih bisa dikirim, katanya ...",1,"barang kadaluarsa masih bisa dikirim, katanya ..."
6,pilihan yg terbaik untuk anak kos 😃👍🏻,5,pilihan yg terbaik untuk anak kos 😃👍🏻
7,1. Cannot remove profile 2. Cannot delete account,1,1. Cannot remove profile 2. Cannot delete account
...,...,...,...
3909,Customer service is very bad. Every time i cal...,4,Customer service is very bad. Every time i cal...
3912,"Puas konsul ama dokter enditiara, sukses selal...",5,"Puas konsul ama dokter enditiara, sukses selal..."
3913,Bagus bagus bagus,5,Bagus bagus bagus
3914,"Nice dok, thanks",5,"Nice dok, thanks"


In [29]:
from langdetect import detect

test=halodoccleaned.head(10)
halodoccleaned['lang_id'] = halodoccleaned['content_contractions'].apply(lambda x: detect(x))

In [30]:
halodoccleaned

,content,score,content_contractions,lang_id
0,Great app. Selama ini hanya dipakai untuk beli...,5,Great app. Selama ini hanya dipakai untuk beli...,id
1,"i've already uploaded the medicine recipe, why...",1,"i have already uploaded the medicine recipe, w...",en
3,"barang kadaluarsa masih bisa dikirim, katanya ...",1,"barang kadaluarsa masih bisa dikirim, katanya ...",id
6,pilihan yg terbaik untuk anak kos 😃👍🏻,5,pilihan yg terbaik untuk anak kos 😃👍🏻,id
7,1. Cannot remove profile 2. Cannot delete account,1,1. Cannot remove profile 2. Cannot delete account,it
...,...,...,...,...
3909,Customer service is very bad. Every time i cal...,4,Customer service is very bad. Every time i cal...,en
3912,"Puas konsul ama dokter enditiara, sukses selal...",5,"Puas konsul ama dokter enditiara, sukses selal...",id
3913,Bagus bagus bagus,5,Bagus bagus bagus,tl
3914,"Nice dok, thanks",5,"Nice dok, thanks",en


In [32]:
halodoccleaned['lang_id'].value_counts()

id    1930
en     624
tl      28
no      14
de      11
so      10
ca       9
da       9
fr       8
ro       7
it       6
af       6
nl       4
et       4
sv       3
sl       3
hr       3
tr       2
sw       2
cy       2
es       2
vi       1
pt       1
sq       1
lt       1
fi       1
Name: lang_id, dtype: int64

In [34]:
halodoctranslated=pd.read_csv(r'data_halodoctranslated.csv')
halodoctranslated['lang_id'].value_counts()

id     1991
en      648
ms       37
su        6
jv        4
hi        2
bg        1
nl        1
fil       1
te        1
Name: lang_id, dtype: int64

In [39]:
halodocfinal = halodoctranslated.loc[halodoctranslated['lang_id'] == "en"]
halodocfinal = halodocfinal.drop(columns=["content","content_translated",'lang_id'])
halodocfinal.to_csv('data_halodocfinal.csv',index=False)
halodocfinal

,score,content_contractions
1,1,"i have already uploaded the medicine recipe, w..."
4,1,1. Cannot remove profile 2. Cannot delete account
5,3,"Doctor consultation always slow in responding,..."
8,5,excellent. I feel sick on the way to my destin...
13,5,Very easy to use
...,...,...
2684,4,Simple n express n very helfulllll
2686,1,Worst service ever. Halodoc is not ready to ha...
2687,4,Customer service is very bad. Every time i cal...
2690,5,"Nice dok, thanks"


# SENTIMENT ANALYSIS

In [3]:
halodocvader = pd.read_csv(r'data_halodocfinal.csv')
halodocvader['compound'] = [analyzer.polarity_scores(x)['compound'] for x in halodocvader['content_contractions']]
halodocvader['neg'] = [analyzer.polarity_scores(x)['neg'] for x in halodocvader['content_contractions']]
halodocvader['neu'] = [analyzer.polarity_scores(x)['neu'] for x in halodocvader['content_contractions']]
halodocvader['pos'] = [analyzer.polarity_scores(x)['pos'] for x in halodocvader['content_contractions']]


#convert rating
def getrating (compound,positive,negative):
    if compound > 0.001:
        if positive > 0.5:
            return 2
        else:
            return 1
    elif compound < -0.001:
        if negative > 0.5:
            return -2
        else:
            return -1
    elif (compound > -0.001) and (compound < 0.001):
        return 0

halodocvader['vader_rating'] = halodocvader.apply(lambda x: getrating(x.compound,x.pos,x.neg),axis=1)
halodocvader.to_csv('data_halodocvader.csv',index=False)
halodocvader

,score,content_contractions,compound,neg,neu,pos,vader_rating
0,1,"i have already uploaded the medicine recipe, w...",0.0000,0.000,1.000,0.000,0
1,1,1. Cannot remove profile 2. Cannot delete account,0.0000,0.000,1.000,0.000,0
2,3,"Doctor consultation always slow in responding,...",0.0000,0.000,1.000,0.000,0
3,5,excellent. I feel sick on the way to my destin...,0.7256,0.092,0.673,0.235,1
4,5,Very easy to use,0.4927,0.000,0.484,0.516,2
...,...,...,...,...,...,...,...
643,4,Simple n express n very helfulllll,0.0000,0.000,1.000,0.000,0
644,1,Worst service ever. Halodoc is not ready to ha...,-0.1541,0.094,0.817,0.088,-1
645,4,Customer service is very bad. Every time i cal...,-0.5849,0.240,0.760,0.000,-1
646,5,"Nice dok, thanks",0.6908,0.000,0.149,0.851,2


# KEYWORD EXTRACTION

In [4]:
from keybert import KeyBERT
kw_model = KeyBERT()

from keyphrase_vectorizers import KeyphraseCountVectorizer
vectorizer = KeyphraseCountVectorizer(pos_pattern='<N.*>+<N.*>+|<N.*>+<N.*>+<N.*>+|<V.*>*<N.*>+|')

In [5]:
halodockeybert = pd.read_csv(r'data_halodocvader.csv')
halodockeybert['keyword'] = halodockeybert['content_contractions'].apply(lambda x: kw_model.extract_keywords(x, vectorizer=vectorizer))
halodockeybert.to_csv('data_halodockeybert.csv',index=False)
halodockeybert

,score,content_contractions,compound,neg,neu,pos,vader_rating,keyword
0,1,"i have already uploaded the medicine recipe, w...",0.0000,0.000,1.000,0.000,0,"[(medicine recipe, 0.6271), (buying medicine, ..."
1,1,1. Cannot remove profile 2. Cannot delete account,0.0000,0.000,1.000,0.000,0,"[(remove profile, 0.7197), (delete account, 0...."
2,3,"Doctor consultation always slow in responding,...",0.0000,0.000,1.000,0.000,0,"[(doctor consultation, 0.6149), (has insurance..."
3,5,excellent. I feel sick on the way to my destin...,0.7256,0.092,0.673,0.235,1,"[(use halodoc, 0.5905), (halodoc, 0.5196), (me..."
4,5,Very easy to use,0.4927,0.000,0.484,0.516,2,[]
...,...,...,...,...,...,...,...,...
643,4,Simple n express n very helfulllll,0.0000,0.000,1.000,0.000,0,"[(n express n, 0.7333)]"
644,1,Worst service ever. Halodoc is not ready to ha...,-0.1541,0.094,0.817,0.088,-1,"[(hospital cashless facility, 0.5215), (cashle..."
645,4,Customer service is very bad. Every time i cal...,-0.5849,0.240,0.760,0.000,-1,"[(customer service, 0.6121), (time, 0.1102)]"
646,5,"Nice dok, thanks",0.6908,0.000,0.149,0.851,2,"[(nice dok, 0.7436), (thanks, 0.557)]"


In [16]:
features_halodoc = pd.read_csv(r'data_halodockeybert.csv')

data = []

#coba=pd.DataFrame(data)
a=-1
for i in features_halodoc['keyword']:
    res = i.strip('][').split(', (')
    #print(i)
    a+=1
#    coba.insert(0,a)
    for j in range (len(res)):
        data.append({'nomer':a,'word':res[j]})
        #coba.append(res[j])
#pd.DataFrame(coba)
data=pd.DataFrame(data)
data=data.set_index('nomer')
data

,word
nomer,
0,"('medicine recipe', 0.6271)"
0,"'buying medicine', 0.5984)"
1,"('remove profile', 0.7197)"
1,"'delete account', 0.6695)"
2,"('doctor consultation', 0.6149)"
...,...
646,"('nice dok', 0.7436)"
646,"'thanks', 0.557)"
647,"('health specialist', 0.5373)"


In [17]:
features_halodoc=pd.merge(features_halodoc,data,left_index=True,right_index=True)
features_halodoc.to_csv('features_halodoc.csv',index=False)
features_halodoc.to_csv('data_features_halodoc.csv',index=False)
features_halodoc

,score,content_contractions,compound,neg,neu,pos,vader_rating,keyword,word
0,1,"i have already uploaded the medicine recipe, w...",0.0000,0.0,1.000,0.000,0,"[('medicine recipe', 0.6271), ('buying medicin...","('medicine recipe', 0.6271)"
0,1,"i have already uploaded the medicine recipe, w...",0.0000,0.0,1.000,0.000,0,"[('medicine recipe', 0.6271), ('buying medicin...","'buying medicine', 0.5984)"
1,1,1. Cannot remove profile 2. Cannot delete account,0.0000,0.0,1.000,0.000,0,"[('remove profile', 0.7197), ('delete account'...","('remove profile', 0.7197)"
1,1,1. Cannot remove profile 2. Cannot delete account,0.0000,0.0,1.000,0.000,0,"[('remove profile', 0.7197), ('delete account'...","'delete account', 0.6695)"
2,3,"Doctor consultation always slow in responding,...",0.0000,0.0,1.000,0.000,0,"[('doctor consultation', 0.6149), ('has insura...","('doctor consultation', 0.6149)"
...,...,...,...,...,...,...,...,...,...
646,5,"Nice dok, thanks",0.6908,0.0,0.149,0.851,2,"[('nice dok', 0.7436), ('thanks', 0.557)]","('nice dok', 0.7436)"
646,5,"Nice dok, thanks",0.6908,0.0,0.149,0.851,2,"[('nice dok', 0.7436), ('thanks', 0.557)]","'thanks', 0.557)"
647,5,Good enough to chat with the health specialist...,0.7003,0.0,0.674,0.326,1,"[('health specialist', 0.5373), ('cmiw', 0.403...","('health specialist', 0.5373)"
647,5,Good enough to chat with the health specialist...,0.7003,0.0,0.674,0.326,1,"[('health specialist', 0.5373), ('cmiw', 0.403...","'cmiw', 0.4031)"


# RE-SWOT

# EVALUATION